In [16]:
#Get full-text seed articles

#encoding=utf8
from xlrd import open_workbook
import urllib
from Bio import Entrez
from xml.dom import minidom
import xml.etree.ElementTree as ET
import os
import ftplib
import urllib.request
import tarfile
import numpy as np
import pandas as pd

#read seed articles
df = pd.read_excel('../Data/Step1.1_Zika_PMCID_PMID_test.xlsx')
df.head(10)

#get list of seed articles
seed_articles_PMCIDs = df['PMCID']
print (len(seed_articles_PMCIDs))

#get the link to physical files
for PMCID in seed_articles_PMCIDs:
    link = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=" + PMCID
    request = urllib.request.Request(link)
    result = urllib.request.urlopen(request)
    tree = ET.parse(result)
    doc = tree.getroot()
    for child in doc:
        if child.tag == "records":
            article_download_link = ""
            for child_level_2 in child:
                if child_level_2.tag == "record":
                    for child_level_3 in child_level_2:
                        if ("link" in child_level_3.tag):
                            link_format = child_level_3.attrib["format"]
                            if (link_format == "pdf"):
                                article_download_link = child_level_3.attrib["href"]
                                
                            elif (article_download_link=="") and (link_format == "tgz"):
                                article_download_link = child_level_3.attrib["href"]
            print (article_download_link)                    
            if (article_download_link.endswith("pdf")):                    
                urllib.request.urlretrieve(article_download_link, 
                                                       '/Citation_Context_Analysis/Seed_Articles_Fulltext_PDF/' + PMCID + ".pdf")
            elif (article_download_link.endswith("tar.gz")):
                urllib.request.urlretrieve(article_download_link, 
                '/Citation_Context_Analysis/Seed_Articles_ZipFile/' + PMCID + ".tar.gz")
                
                tar = tarfile.open('/Citation_Context_Analysis/Seed_Articles_ZipFile/' + PMCID + ".tar.gz", "r:gz")
                for member in tar.getmembers():
                    if (member.name.endswith("xml")):
                        f = tar.extractfile(member)
                        content = f.read()
                        with open('/Citation_Context_Analysis/Seed_Articles_Fulltext_XML/' + PMCID + '.xml', 'wb') as file:
                            file.write(content)
                    elif (member.name.endswith("pdf")):
                        f = tar.extractfile(member)
                        content = f.read()
                        with open('/Citation_Context_Analysis/Seed_Articles_Fulltext_PDF/' + PMCID + '.pdf', 'wb') as file:
                            file.write(content)

4
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/16/f3/PMC5657812.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c3/ea/PMC5207634.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/61/b2/PMC5996208.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/14/85/PMC4774925.tar.gz
